# Visualizing Public Transportation Speeds with Python
When people choose between driving and public transportation for urban trips, travel time (and public transportation frequency) is the most influential factor in their decision. Here we use Python to evaluate public transportation timetable speed between stops to look for slow spots. First we load some libraries:

In [2]:
import requests
import pandas
import tempfile
import datetime
import gtfstk
import folium
import haversine
from functools import reduce
from multiprocessing import Pool

Next we create some helper functions. We make a function url2gtfs to extend the gtfstk library so it can load a library directly without downloading it first. We create parseTime to deal with date fields that go past 24 hours in timetables. Next we make tripToSegments to split up each public transportation vehicle's "trip" into "segments" between stops. And last we create function to plot each segment.

In [3]:
def url2gtfs(url):
    r = requests.get(url)
    with tempfile.NamedTemporaryFile() as f:
      f.write(r._content)
      return gtfstk.read_gtfs(f.name, dist_units='mi')


def parseTime(time): # to deal with times after midnight
    if int(time.split(':')[0]) > 23:
        return datetime.datetime.strptime('2:' + str(int(time.split(':')[0]) - 24) + ':'
                                          + time.split(':')[1] + ':' + time.split(':')[2], '%d:%H:%M:%S')
    else:
        return datetime.datetime.strptime('1:' + time, '%d:%H:%M:%S')


def tripToSegments(trip_id):
    stop_times = gtfs.stop_times[gtfs.stop_times['trip_id'] == trip_id]
    segments = []
    for i in range(len(stop_times)-1):
        origin = gtfs.stops[gtfs.stops['stop_id'] == stop_times['stop_id'].iloc[i]]
        destination = gtfs.stops[gtfs.stops['stop_id'] == stop_times['stop_id'].iloc[i+1]]
        distance = haversine.haversine((origin['stop_lat'], origin['stop_lon']),
                         (destination['stop_lat'], destination['stop_lon']),
                         miles=True)
        startTime = parseTime(stop_times['departure_time'].iloc[i])
        stopTime = parseTime(stop_times['arrival_time'].iloc[i+1])
        duration = (stopTime - startTime).seconds
        if duration == 0:
            speed = 0 # when the laws of physics do not apply
        else:
            speed = distance / duration * 60 * 60
        segments.append({'origin_id': origin['stop_id'].iloc[0],
                         'destination_id': destination['stop_id'].iloc[0],
                         'distance': distance,
                         'duration': duration,
                         'speed': speed})
    return(segments)

def plotSegment(segment):
    origin = gtfs.stops[gtfs.stops['stop_id'] == segment.name[0]]
    destination = gtfs.stops[gtfs.stops['stop_id'] == segment.name[1]]
    folium.PolyLine(locations=[(origin.stop_lat.iloc[0], origin.stop_lon.iloc[0]),
                               (destination.stop_lat.iloc[0], destination.stop_lon.iloc[0])],
                    popup=str(segment.speed['min']),
                    weight=segment.speed['min']
                   ).add_to(foliumMap)

Here we specify the General Transit Feed Specification (GTFS) feed for the public transportation agency we want to analyze. Then we can look at the routes in that feed.

In [4]:
url = 'https://gitlab.com/LACMTA/gtfs_rail/raw/master/gtfs_rail.zip' # Los Angeles County Metro
# url = 'http://github.com/transitland/gtfs-archives-not-hosted-elsewhere/raw/master/amtrak.zip' # Amtrak
gtfs = url2gtfs(url)

In [5]:
gtfs.routes

,route_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_url
0,801,NaN,Metro Blue Line (801),NaN,0,004DAC,FFFFFF,http://www.metro.net/around/rail/blue-line/
1,802,NaN,Metro Red Line (802),NaN,1,EE3A43,FFFFFF,http://www.metro.net/around/rail/red-line/
2,803,NaN,Metro Green Line (803),NaN,0,2EAB00,FFFFFF,http://www.metro.net/around/rail/green-line/
3,804,NaN,Metro Gold Line (804),NaN,0,DA7C20,FFFFFF,http://www.metro.net/around/rail/gold-line/
4,806,NaN,Metro Expo Line (806),NaN,0,0177A5,FFFFFF,http://www.metro.net/around/rail/expo-line/
5,805,NaN,Metro Purple Line (805),NaN,1,9561A9,FFFFFF,http://www.metro.net/around/rail/purple-line/


Next we pick the route we want to analyse and the direction (0.0 or 1.0). We filter the GTFS data for the route and direction, and split the trips into segments. Here we have the min, max, mean speed between each stop pair.

In [7]:
route_id = '806'
direction_id = 1.0

trip_ids = gtfs.trips[(gtfs.trips['route_id'] == route_id) & (gtfs.trips['direction_id'] == direction_id)]['trip_id']
p = Pool(8)
segments = reduce(lambda x, y: x + y, p.map(tripToSegments, trip_ids))
segmentsDF = pandas.DataFrame(segments).groupby(['origin_id', 'destination_id']).describe()
segmentsDF

distance                                              \
                            count      mean           std       min       25%   
origin_id destination_id                                                        
80121     80123            1021.0  0.910273  2.221534e-16  0.910273  0.910273   
80122     80121            1021.0  0.685952  1.110767e-16  0.685952  0.685952   
80123     80124            1021.0  0.547751  1.110767e-16  0.547751  0.547751   
80124     80125            1021.0  0.512530  1.110767e-16  0.512530  0.512530   
80125     80126            1021.0  0.332504  5.553836e-17  0.332504  0.332504   
80126     80127            1021.0  0.994773  4.443068e-16  0.994773  0.994773   
80127     80128            1021.0  1.526340  2.221534e-16  1.526340  1.526340   
80128     80129            1021.0  0.638851  0.000000e+00  0.638851  0.638851   
80129     80130            1021.0  0.522143  0.000000e+00  0.522143  0.522143   
80130     80131            1021.0  0.977151  2.221534e-16  0.977151  0.977151   
80131     80132            1021.0  0.971643  2.221534e-16  0.971643  0.971643   
80132     80133            1021.0  0.879870  3.332301e-16  0.879870  0.879870   
80133     80134            1021.0  1.273532  2.221534e-16  1.273532  1.273532   
80134     80135            1021.0  0.561476  2.221534e-16  0.561476  0.561476   
80135     80136            1021.0  1.098758  2.221534e-16  1.098758  1.098758   
80136     80137             999.0  0.963716  1.110779e-16  0.963716  0.963716   
80137     80138             999.0  0.725943  1.110779e-16  0.725943  0.725943   
80138     80139             999.0  0.892721  0.000000e+00  0.892721  0.892721   

                                                       duration              \
                               50%       75%       max    count        mean   
origin_id destination_id                                                      
80121     80123           0.910273  0.910273  0.910273   1021.0  230.401567   
80122     80121           0.685952  0.685952  0.685952   1021.0  177.747307   
80123     80124           0.547751  0.547751  0.547751   1021.0  170.401567   
80124     80125           0.512530  0.512530  0.512530   1021.0  110.401567   
80125     80126           0.332504  0.332504  0.332504   1021.0  110.401567   
80126     80127           0.994773  0.994773  0.994773   1021.0  170.401567   
80127     80128           1.526340  1.526340  1.526340   1021.0  230.401567   
80128     80129           0.638851  0.638851  0.638851   1021.0  110.401567   
80129     80130           0.522143  0.522143  0.522143   1021.0  110.401567   
80130     80131           0.977151  0.977151  0.977151   1021.0  110.401567   
80131     80132           0.971643  0.971643  0.971643   1021.0  139.196866   
80132     80133           0.879870  0.879870  0.879870   1021.0  110.401567   
80133     80134           1.273532  1.273532  1.273532   1021.0  230.401567   
80134     80135           0.561476  0.561476  0.561476   1021.0  110.401567   
80135     80136           1.098758  1.098758  1.098758   1021.0  170.401567   
80136     80137           0.963716  0.963716  0.963716    999.0  110.390390   
80137     80138           0.725943  0.725943  0.725943    999.0   50.390390   
80138     80139           0.892721  0.892721  0.892721    999.0  240.000000   

                            ...                     speed             \
                            ...        75%    max   count       mean   
origin_id destination_id    ...                                        
80121     80123             ...      240.0  240.0  1021.0  14.249816   
80122     80121             ...      180.0  240.0  1021.0  14.122148   
80123     80124             ...      180.0  180.0  1021.0  11.612220   
80124     80125             ...      120.0  120.0  1021.0  16.851746   
80125     80126             ...      120.0  120.0  1021.0  10.932578   
80126     80127             ...      180.0  180.0  1021.0  21.089003   
80127     80128

And finally we plot the route where the width of the line is the speed between each stop pair. The Expo Line really is much slower into downtown than along the rest of the line.

In [8]:
foliumMap = folium.Map(location=[34, -118], zoom_start=9)
segmentsDF.apply(plotSegment, axis=1)
foliumMap